This is a notebook to examine annotations for spontaneous behaviors.  It is not involved in processing data for any publication results, but is meant only for data exploration. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import itertools
from pathlib import Path

import pandas as pd

from keller_zlatic_vnc.data_processing import apply_quiet_and_cutoff_times
from keller_zlatic_vnc.data_processing import generate_standard_id_for_full_annots
from keller_zlatic_vnc.data_processing import get_basic_clean_annotations_from_full
from keller_zlatic_vnc.data_processing import read_full_annotations

/Users/bishopw/Documents/Janelia_Research/Projects/janelia_core/janelia_core/fileio/exp_reader.py:21: UserWarning: Unable to locate pyklb module.  Will not be able to read in .klb files.
  warnings.warn('Unable to locate pyklb module.  Will not be able to read in .klb files.')


## Parameters go here 

In [3]:
ps = dict()

# Give locations of folders with annotations
ps['annot_folders'] = [r'/Users/bishopw/Desktop/full_annotations/behavior_csv_cl_A4', 
                       r'/Users/bishopw/Desktop/full_annotations/behavior_csv_cl_A9']

# Specify the definition for clean events we use 
ps['clean_event_def'] = 'po'

# Specify the threshold we use (in number of stacks) to determine when a quiet transition has occurred
ps['q_th'] = 55

# Specify the cut off threshold we use (in number of stacks) to determine when a real transition has occurred
ps['co_th'] = 3

# Specify the length of events we mark new quiet events as 
ps['q_length'] = 3

## Get list of all subjects we have annotations for 

In [4]:
annot_file_paths = list(itertools.chain(*[glob.glob(str(Path(folder) / '*.csv')) for folder in ps['annot_folders']]))
annot_file_names = [Path(p).name for p in annot_file_paths]
annot_subjs = [generate_standard_id_for_full_annots(fn) for fn in annot_file_names]


## Read in basic annotations for all subjects 

In [5]:
annotations = []
for s_id, s_annot_file in zip(annot_subjs, annot_file_paths):
    tbl = read_full_annotations(s_annot_file)
    tbl['subject_id'] = s_id
    annotations.append(tbl)

## Filter down to only "clean" events 

In [6]:
annotations = [get_basic_clean_annotations_from_full(annot, clean_def=ps['clean_event_def'])
                   for annot in annotations]

## Put all events into the same table 

In [7]:
annotations = pd.concat(annotations, ignore_index=True)

## Mark preceding and succeeding behaviors as known, quiet or unknown 

In [8]:
annotations = apply_quiet_and_cutoff_times(annotations, quiet_th=ps['q_th'], co_th=ps['co_th'], q_length=ps['q_length'])

## View annotations

In [9]:
annotations[annotations['beh_after'] == 'Q']

,start,end,beh,subject_id,beh_before,beh_before_start,beh_before_end,beh_after,beh_after_start,beh_after_end
29,1474,1490,F,CW_17-08-27-L2,U,1457,1460,Q,2069,2071
54,3153,3176,F,CW_17-08-27-L2,U,3116,3134,Q,4161,4163
118,887,912,F,CW_17-11-26-L4,F,864,884,Q,1728,1730
132,2911,2925,F,CW_17-11-26-L4,U,2878,2892,Q,3627,3629
188,579,590,F,CW_17-11-27-L3,U,558,568,Q,1443,1445
...,...,...,...,...,...,...,...,...,...,...
6662,176,184,B,CW_17-11-03-L3,U,167,172,Q,1063,1065
6693,2513,2522,B,CW_17-11-03-L3,U,2502,2509,Q,3091,3093
6713,4064,4072,P,CW_17-11-03-L3,F,4033,4061,Q,4597,4599
6738,664,683,P,CW_17-11-07-L4,U,634,659,Q,1222,1224
